In [1]:
import os  
import pdfplumber
import numpy as np
import pandas as pd
from WindPy import w


In [2]:

w.start() # 默认命令超时时间为120秒，如需设置超时时间可以加入waitTime参数，例如waitTime=60,即设置命令超时时间为60秒 

w.isconnected() # 判断WindPy是否已经登录成功


Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2017 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


True

In [3]:
#改金额单位
def mc(amount):
    value = ''
    amount = float(amount)
    if abs(amount) >= 10000 and abs(amount) < 100000000:
        val = round(amount/10000)
        value = str(val) + "万元"
    elif abs(amount) >= 100000000:
        val = round(amount/100000000,2)
        value = str(val) + "亿元"
        
    return value 

In [4]:
def file_name(file_dir):  
    L=[]  
    for root, dirs, files in os.walk(file_dir): 
        for file in files: 
            if os.path.splitext(file)[1] == '.pdf': 
                L.append(os.path.join(root, file)) 
    return L


In [5]:
# 读取每一个pdf的股票代码
dir_name = 'D:/GF/time_saver/半年度报告生成器'
# dir_list = os.listdir(dir_name)
# print(dir_list[1])
filelist = file_name(dir_name)


tot_num_files = np.size(filelist)
code = {}
#
for i in range(0,tot_num_files-1):
    if filelist[i].find('.S')!= -1 :
        a = filelist[i].find('.S') + 3
        b = filelist[i][:a]
        c = b[(a-9):]
        code[i] = c
        

        


filelist

['D:/GF/time_saver/半年度报告生成器\\20190821-300085.SZ-银之杰：2019年半年度报告.pdf',
 'D:/GF/time_saver/半年度报告生成器\\2020-07-29-002296.SZ-辉煌科技_2020年半年度报告.pdf',
 'D:/GF/time_saver/半年度报告生成器\\2020-07-29-600519.SH-600519贵州茅台2020年半年度报告.pdf',
 'D:/GF/time_saver/半年度报告生成器\\2020-07-29-688396.SH-688396华润微2020年半年度报告.pdf',
 'D:/GF/time_saver/半年度报告生成器\\2020-08-04-002439.SZ-启明星辰_2020年半年度报告.pdf']

In [6]:
filename = "2020-07-29-002296.SZ-辉煌科技_2020年半年度报告.pdf"
code = input("股票代码：")
pdf = pdfplumber.open(filename)


股票代码：002296.SZ


In [7]:
############################ 主要会计数据和财务指标 ###############################

# 自动查找页码

for i in range(2,10):
    p0 = pdf.pages[i]
    pp = p0.extract_text(x_tolerance=3, y_tolerance=3)
    if pp.find("主要会计数据")> -1:
        page_num = i
        break
        
        

In [8]:


p0 = pdf.pages[page_num]
table = p0.extract_table()


In [9]:
table

[['', '本报告期', '上年同期', '本报告期比上年同期增减'],
 ['营业收入（元）', '195,611,258.46', '204,278,084.75', '-4.24%'],
 ['归属于上市公司股东的净利润（元）', '45,460,145.89', '35,179,031.40', '29.23%'],
 ['归属于上市公司股东的扣除非经常性损', '', '', ''],
 [None, '43,070,752.81', '27,620,823.37', '55.94%'],
 ['益的净利润（元）', None, None, None],
 [None, '', '', ''],
 ['经营活动产生的现金流量净额（元）', '-34,866,512.83', '27,518,354.57', '-226.70%'],
 ['基本每股收益（元/股）', '0.1204', '0.0934', '28.91%'],
 ['稀释每股收益（元/股）', '0.1204', '0.0934', '28.91%'],
 ['加权平均净资产收益率', '2.93%', '2.38%', '0.55%'],
 ['', '本报告期末', '上年度末', '本报告期末比上年度末增减'],
 ['总资产（元）', '1,871,464,151.83', '1,867,276,002.10', '0.22%'],
 ['归属于上市公司股东的净资产（元）', '1,553,409,098.03', '1,525,653,331.96', '1.82%']]

In [10]:

df = pd.DataFrame(table[1:6], columns=table[0])
# for column in ["Effective", "Received"]:
#     df[column] = df[column].str.replace(" ", "")
df



,,本报告期,上年同期,本报告期比上年同期增减
0,营业收入（元）,"195,611,258.46","204,278,084.75",-4.24%
1,归属于上市公司股东的净利润（元）,"45,460,145.89","35,179,031.40",29.23%
2,归属于上市公司股东的扣除非经常性损,,,
3,None,"43,070,752.81","27,620,823.37",55.94%
4,益的净利润（元）,None,None,None


In [11]:
# 洗数据。。。
data = df.values

#设置变量index，保证扣非的读取
test_var1 = data[3,0]
test_var2 = data[2,0]
index = []
if test_var1!= None and test_var2.find("经常性")!= -1:
    index = int(2)
elif test_var1==None and test_var2.find("经常性")!= -1:
    index = int(3)
    
    
percent = {}

#去除逗号
for i in [0,1,index]:
    for j in [1,2,3]:
        data[i,j] = data[i,j].replace(',','')
        percent[i] = data[i,3]
        if percent[i].find('%')!=-1:
            percent[i] = percent[i].replace('%','')
        



In [12]:
# 修改数据类型
percent[0] = float(percent[0])
percent[1] = float(percent[1])
percent[index] = float(percent[index])

# 判断增减
updown = {}
for i in [0,1,index]:
    if percent[i]>=0:
        updown[i]="增加"
    else:
        updown[i]="减少"
        percent[i]=abs(percent[i])



In [13]:
################################ 营业利润 #########################################
# 从合并利润表里提取
page_num1 = 0
for z in range(20,80):
    p1 = pdf.pages[z]
    pp1 = p1.extract_text(x_tolerance=3, y_tolerance=3)
    if pp1.find("三、营业利润（亏损")> -1:
        page_num1 = z
        break
        

# 提取表格内容
p1 = pdf.pages[page_num1]
table1 = p1.extract_table()
table1


[['“－”号填列）', '', ''],
 ['信用减值损失（损失以“-”号填', '', ''],
 [None, '-3,911,592.87', '-2,777,353.40'],
 ['列）', None, None],
 [None, '', ''],
 ['资产减值损失（损失以“-”号填', '', ''],
 [None, '', ''],
 ['列）', None, None],
 [None, '', ''],
 ['资产处置收益（损失以“-”号填', '', ''],
 [None, '', ''],
 ['列）', None, None],
 [None, '', ''],
 ['三、营业利润（亏损以“－”号填列）', '51,530,970.75', '39,763,071.73'],
 ['加：营业外收入', '659,467.37', '36,568.90'],
 ['减：营业外支出', '1,642,665.31', '486,683.51'],
 ['四、利润总额（亏损总额以“－”号填列）', '50,547,772.81', '39,312,957.12'],
 ['减：所得税费用', '7,429,100.31', '4,807,371.86'],
 ['五、净利润（净亏损以“－”号填列）', '43,118,672.50', '34,505,585.26'],
 ['（一）按经营持续性分类', '', ''],
 ['1.持续经营净利润（净亏损以“－”', '', ''],
 [None, '43,118,672.50', '34,505,585.26'],
 ['号填列）', None, None],
 [None, '', ''],
 ['2.终止经营净利润（净亏损以“－”', '', ''],
 [None, '', ''],
 ['号填列）', None, None],
 [None, '', ''],
 ['（二）按所有权归属分类', '', ''],
 ['1.归属于母公司所有者的净利润', '45,460,145.89', '35,179,031.40'],
 ['2.少数股东损益', '-2,341,473.39', '-673,446.14'],
 ['六、其他综合收益的税后净额', '', ''],
 ['

In [14]:
# 把表格装进dataframe
df1 = pd.DataFrame(table1, columns=table1[0])

df1


,“－”号填列）,,
0,“－”号填列）,,
1,信用减值损失（损失以“-”号填,,
2,None,"-3,911,592.87","-2,777,353.40"
3,列）,None,None
4,None,,
...,...,...,...
56,5.其他,,
57,（二）将重分类进损益的其他综合,,
58,None,,
59,收益,None,None


In [15]:
# 还是洗数据

#查找营业利润在第几行：

dfsize = df1.count()[0]

for i in range(0,dfsize-1):
    if df1.iloc[i][0] == "三、营业利润（亏损以“－”号填列）":
        loc = i
        break

# 读取该行数据
# 分离附注

profit_this_temp = df1.iloc[loc][1]
profit_last_temp = df1.iloc[loc][2]

# 去逗号
profit_this = float(profit_this_temp.replace(',',''))
profit_last = float(profit_last_temp.replace(',',''))

# 计算增减比率
profit_var = round(100*(profit_this - profit_last)/profit_last,2)


# 判断增减

if profit_var > 0:
    updown_p = '增加'
elif profit_var < 0:
    updown_p = '减少'

#特殊情况
if profit_this < 0 and profit_this < 0:
    updown_p = '减少'
    
    

In [16]:
Revenue = {'obj':data[0,0],'this':data[0,1],'last':data[0,2],'varpct':percent[0]} #把数据写入dict

NetProfit_sh = {'obj':data[1,0],'this':data[1,1],'last':data[1,2],'varpct':percent[1]} #把数据写入dict

NetProfit_sh_loss = {'obj':data[2,0],'this':data[index,1],'last':data[index,2],'varpct':percent[index]} #把数据写入dict

Profit = {'obj':df1.iloc[loc][0],'this':profit_this,'last':profit_last,'varpct':profit_var}




In [17]:
# 上半年生成
line1_h1 = "20H1营业总收入" + mc(Revenue['this']) + ",上年同期为" + mc(Revenue['last']) + ",同比" + updown[0] + str(percent[0]) + '%'
line2_h1 = "20H1营业利润" + mc(Profit['this']) + ",上年同期为" + mc(Profit['last']) + ",同比" + updown_p + str(Profit['varpct']) + '%'
line3_h1 = "20H1归母净利润" + mc(NetProfit_sh['this']) + ",上年同期为" + mc(NetProfit_sh['last']) + ",同比" + updown[1] + str(percent[1]) + '%'
line4_h1 = "20H1扣非归母净利润" + mc(NetProfit_sh_loss['this']) + ",上年同期为" + mc(NetProfit_sh_loss['last']) + ",同比" + updown[index] + str(percent[index]) + '%'

In [18]:
######### 以下是wind部分 ##########

In [19]:
#营业总收入 _y
# 从wind查找q1数据

wsddata1 = w.wsd(code, "tot_oper_rev", "2019-01-01", "2020-03-31", "rptType=1;Period=Q;Days=Alldays;Fill=Previous")
wsddata1


.ErrorCode=0
.Codes=[002296.SZ]
.Fields=[TOT_OPER_REV]
.Times=[20190331,20190630,20190930,20191231,20200331]
.Data=[[85068095.65,204278084.75,296281288.63,504408910.17,59765104.01]]

In [20]:
#计算q2数据
lastq1_y = wsddata1.Data[0][0] #选取2019年q1数据
lasth1_y = wsddata1.Data[0][1] #选取2019年h1数据
thisq1_y = wsddata1.Data[0][4] #选取2020年q1数据
thish1_y = float(Revenue['this'])
lastq2_y = lasth1_y - lastq1_y    #计算2019年q2数据
thisq2_y = thish1_y - thisq1_y 

#计算同比增长/减少
growth_year = {}
growth_year[0] = (thisq2_y - lastq2_y)/abs(lastq2_y) * 100

In [21]:
#归母净利润 _g
# 从wind查找q1数据
wsddata2 = w.wsd(code, "np_belongto_parcomsh", "2019-01-01", "2020-03-31", "rptType=1;Period=Q;Days=Alldays;Fill=Previous")
wsddata2



.ErrorCode=0
.Codes=[002296.SZ]
.Fields=[NP_BELONGTO_PARCOMSH]
.Times=[20190331,20190630,20190930,20191231,20200331]
.Data=[[5262503.48,35179031.4,58059559.69,67034018.98,12745831.46]]

In [22]:

#计算q2数据
lastq1_g = wsddata2.Data[0][0] #选取2019年q1数据
lasth1_g = wsddata2.Data[0][1] #选取2019年h1数据
thisq1_g = wsddata2.Data[0][4] #选取2020年q1数据
thish1_g = float(NetProfit_sh['this'])
lastq2_g = lasth1_g - lastq1_g     #计算2019年q2数据
thisq2_g = thish1_g - thisq1_g 

#计算同比增长/减少
growth_year[1] = (thisq2_g - lastq2_g)/abs(lastq2_g) * 100

In [23]:
#扣非归母净利润 _k
# 从wind查找q1数据
wsddata3 = w.wsd(code, "deductedprofit", "2019-01-01", "2020-03-31", "rptType=1;Period=Q;Days=Alldays;Fill=Previous")
wsddata3


.ErrorCode=0
.Codes=[002296.SZ]
.Fields=[DEDUCTEDPROFIT]
.Times=[20190331,20190630,20190930,20191231,20200331]
.Data=[[4109332.65,27620823.37,50139481.71,67724411.54,11187867.25]]

In [24]:
#计算q2数据
lastq1_k = wsddata3.Data[0][0] #选取2019年q1数据
lasth1_k = wsddata3.Data[0][1] #选取2019年h1数据
thisq1_k = wsddata3.Data[0][4] #选取2020年q1数据
thish1_k = float(NetProfit_sh_loss['this'])
lastq2_k = lasth1_k - lastq1_k     #计算2019年q2数据
thisq2_k = thish1_k - thisq1_k 

#计算同比增长/减少
growth_year[2] = (thisq2_k - lastq2_k)/abs(lastq2_k) * 100

In [25]:
#营业利润 _lr
# 从wind查找q1数据
wsddata4 = w.wsd(code, "opprofit", "2019-01-01", "2020-03-31", "rptType=1;Period=Q;Days=Alldays;Fill=Previous")
wsddata4


.ErrorCode=0
.Codes=[002296.SZ]
.Fields=[OPPROFIT]
.Times=[20190331,20190630,20190930,20191231,20200331]
.Data=[[6683600.24,39763071.73,61308429.15,75386956.65,13915865.41]]

In [26]:
#计算q2数据
lastq1_lr = wsddata4.Data[0][0] #选取2019年q1数据
lasth1_lr = wsddata4.Data[0][1] #选取2019年h1数据
thisq1_lr = wsddata4.Data[0][4] #选取2020年q1数据
thish1_lr = float(Profit['this'])
lastq2_lr = lasth1_lr - lastq1_lr     #计算2019年q2数据
thisq2_lr = thish1_lr - thisq1_lr 

#计算同比增长/减少
growth_year_p = (thisq2_lr - lastq2_lr)/abs(lastq2_lr) * 100

In [27]:
# 判断增减
updown_q2 = {}
for i in [0,1,2]:
    if growth_year[i]>=0:
        updown_q2[i]="增加"
    else:
        updown_q2[i]="减少"
        growth_year[i]=abs(growth_year[i])

if growth_year_p >=0:
    updown_p_q2="增加"
else:
    updown_p_q2="减少"



In [28]:
# 二季度生成
line1_q2 = "20Q2营业总收入" + mc(thisq2_y) + ",上年同期为" + mc(lastq2_y) + ",同比" + updown_q2[0] + str(round(growth_year[0],2)) + '%'
line2_q2 = "20Q2营业利润" + mc(thisq2_lr) + ",上年同期为" + mc(lastq2_lr) + ",同比" + updown_p_q2 + str(round(growth_year_p,2)) + '%'
line3_q2 = "20Q2归母净利润" + mc(thisq2_g) + ",上年同期为" + mc(lastq2_g) + ",同比" + updown_q2[1] + str(round(growth_year[1],2)) + '%'
line4_q2 = "20Q2扣非归母净利润" + mc(thisq2_k) + ",上年同期为" + mc(lastq2_k) + ",同比" + updown_q2[2] + str(round(growth_year[2],2)) + '%'

In [29]:
############################### 费用 ################################

# 从合并利润表里提取
page_num2 = 0
for y in range(20,80):
    p2 = pdf.pages[y]
    pp2 = p2.extract_text(x_tolerance=3, y_tolerance=3)
    if pp2.find("研发费用")> -1:
        page_num2 = y
        break
        

# 提取表格内容
p2 = pdf.pages[page_num2]
table2 = p2.extract_table()
table2


[['项目', '2020年半年度', '2019年半年度'],
 ['一、营业总收入', '195,611,258.46', '204,278,084.75'],
 ['其中：营业收入', '195,611,258.46', '204,278,084.75'],
 ['利息收入', '', ''],
 ['已赚保费', '', ''],
 ['手续费及佣金收入', '', ''],
 ['二、营业总成本', '156,253,790.92', '174,716,821.03'],
 ['其中：营业成本', '101,866,754.47', '109,762,369.86'],
 ['利息支出', '', ''],
 ['手续费及佣金支出', '', ''],
 ['退保金', '', ''],
 ['赔付支出净额', '', ''],
 ['提取保险责任准备金净额', '', ''],
 ['保单红利支出', '', ''],
 ['分保费用', '', ''],
 ['税金及附加', '1,929,863.14', '3,208,133.43'],
 ['销售费用', '11,273,153.83', '15,095,688.89'],
 ['管理费用', '14,958,923.89', '15,215,576.49'],
 ['研发费用', '28,547,960.31', '28,625,149.95'],
 ['财务费用', '-2,322,864.72', '2,809,902.41'],
 ['其中：利息费用', '137,946.02', '3,777,312.13'],
 ['利息收入', '2,624,257.76', '1,496,728.62'],
 ['加：其他收益', '19,335,162.73', '12,575,402.29'],
 ['投资收益（损失以“－”号填', '', ''],
 [None, '-3,250,066.65', '403,759.12'],
 ['列）', None, None],
 [None, '', ''],
 ['其中：对联营企业和合营企业', '', ''],
 [None, '-4,904,514.42', '-2,359,749.87'],
 ['的投资收益', None, None],
 

In [30]:
# 把表格装进dataframe
df2 = pd.DataFrame(table2, columns=table2[0])

df2


,项目,2020年半年度,2019年半年度
0,项目,2020年半年度,2019年半年度
1,一、营业总收入,"195,611,258.46","204,278,084.75"
2,其中：营业收入,"195,611,258.46","204,278,084.75"
3,利息收入,,
4,已赚保费,,
5,手续费及佣金收入,,
6,二、营业总成本,"156,253,790.92","174,716,821.03"
7,其中：营业成本,"101,866,754.47","109,762,369.86"
8,利息支出,,
9,手续费及佣金支出,,


In [31]:
# 还是洗数据

#查找费用在第几行：

dfsize2 = df2.count()[0]

for i in range(0,dfsize2-1):
    if df2.iloc[i][0] == "销售费用":
        loc2 = i
        break

# 填写数据+去逗号
mktex_this = float(df2.iloc[loc2][1].replace(',',''))
adminex_this = float(df2.iloc[loc2+1][1].replace(',',''))
rndex_this = float(df2.iloc[loc2+2][1].replace(',',''))
finex_this = float(df2.iloc[loc2+3][1].replace(',',''))

mktex_last = float(df2.iloc[loc2][2].replace(',',''))
adminex_last = float(df2.iloc[loc2+1][2].replace(',',''))
rndex_last = float(df2.iloc[loc2+2][2].replace(',',''))
finex_last = float(df2.iloc[loc2+3][2].replace(',',''))

# 计算同比增减

mktex_var = round(100*(mktex_this - mktex_last)/mktex_last,2)
adminex_var = round(100*(adminex_this - adminex_last)/adminex_last,2)
rndex_var = round(100*(rndex_this - rndex_last)/rndex_last,2)
finex_var = round(100*(finex_this - finex_last)/finex_last,2)


# 判断增减
updown_ex={}
numm = 0
for i in [mktex_var,adminex_var,rndex_var,finex_var]:    
    if i > 0:
        updown_ex[numm] = '增加'
    elif i < 0:
        updown_ex[numm] = '减少'
    numm = numm+1


# 计算今年的费用率
mktex_rate_this = round(100*mktex_this/float(Revenue['this']),2)
adminex_rate_this = round(100*adminex_this/float(Revenue['this']),2)
rndex_rate_this = round(100*rndex_this/float(Revenue['this']),2)
finex_rate_this = round(100*finex_this/float(Revenue['this']),2)

# 计算去年的费用率
mktex_rate_last = round(100*mktex_last/float(Revenue['last']),2)
adminex_rate_last = round(100*adminex_last/float(Revenue['last']),2)
rndex_rate_last = round(100*rndex_last/float(Revenue['last']),2)
finex_rate_last = round(100*finex_last/float(Revenue['last']),2)

# 计算差值
mktex_rate_diff = round(mktex_rate_this - mktex_rate_last,2)
adminex_rate_diff = round(adminex_rate_this - adminex_rate_last,2)
rndex_rate_diff = round(rndex_rate_this - rndex_rate_last,2)
finex_rate_diff = round(finex_rate_this - finex_rate_last,2)

# 判断较去年同期增减
updown_rate={}
nu = 0
for i in [mktex_rate_diff,adminex_rate_diff,rndex_rate_diff,finex_rate_diff]:    
    if i > 0:
        updown_rate[nu] = '增加'
    elif i < 0:
        updown_rate[nu] = '减少'
    nu = nu+1


In [32]:
rateline1 = "销售费用" + mc(mktex_this) + "，同比" + updown_ex[0] + str(abs(mktex_var)) + '%' +"；销售费用率"+ str(mktex_rate_this)+'%'+"，较去年同期"+ updown_rate[0] + str(abs(mktex_rate_diff)) + 'pct；'
rateline2 = "管理费用" + mc(adminex_this) + "，同比" + updown_ex[1] + str(abs(adminex_var)) + '%' +"；管理费用率"+ str(adminex_rate_this)+'%'+"，较去年同期"+ updown_rate[1] + str(abs(adminex_rate_diff)) + 'pct；'
rateline3 = "研发费用" + mc(rndex_this) + "，同比" + updown_ex[2] + str(abs(rndex_var)) + '%' +"；研发费用率"+ str(rndex_rate_this)+'%'+"，较去年同期"+ updown_rate[2] + str(abs(rndex_rate_diff)) + 'pct；'
rateline4 = "财务费用" + mc(finex_this) + "，同比" + updown_ex[3] + str(abs(finex_var)) + '%' +"；财务费用率"+ str(finex_rate_this)+'%'+"，较去年同期"+ updown_rate[3] + str(abs(finex_rate_diff)) + 'pct。'

In [33]:
############################### 现金流 ################################

# 从主营业务分析表里提取
page_num3_a = 0
page_num3_b = 0
page_num3_c = 0
countera = 0
counterb = 0
counterc = 0
for w in range(20,75):
    p3 = pdf.pages[w]
    pp3 = p3.extract_text(x_tolerance=3, y_tolerance=3)
    if pp3.find("经营活动产生的现金流量净额")> -1:
        countera = countera + 1
        if countera == 1:
            page_num3_a = w
    if pp3.find("投资活动产生的现金流量净额")> -1:
        counterb = counterb + 1
        if counterb == 1:
            page_num3_b = w
        
    if pp3.find("筹资活动产生的现金流量净额")> -1:
        counterc = counterc + 1
        if counterc == 1:
            page_num3_c = w
      
        
# 寻找需要提取表格的页码
if page_num3_a == page_num3_b and page_num3_b != page_num3_c: # 如果投资活动现金流和经营现金流在同一页
    g = True
else: # 如果不在同一页（投资和筹资在一起）
    g = False
    

# 提取表格内容
p3a = pdf.pages[page_num3_a]
table3a = p3a.extract_table()
table3a

# print(page_num3_a)
# print(page_num3_b)
# print(page_num3_c)


[['代理买卖证券收到的现金净额', '', ''],
 ['收到的税费返还', '17,195,264.15', '6,063,582.14'],
 ['收到其他与经营活动有关的现金', '7,289,149.09', '13,191,266.55'],
 ['经营活动现金流入小计', '229,205,393.59', '276,493,987.39'],
 ['购买商品、接受劳务支付的现金', '165,288,624.40', '120,233,904.88'],
 ['客户贷款及垫款净增加额', '', ''],
 ['存放中央银行和同业款项净增加', '', ''],
 [None, '', ''],
 ['额', None, None],
 [None, '', ''],
 ['支付原保险合同赔付款项的现金', '', ''],
 ['拆出资金净增加额', '', ''],
 ['支付利息、手续费及佣金的现金', '', ''],
 ['支付保单红利的现金', '', ''],
 ['支付给职工以及为职工支付的现', '', ''],
 [None, '54,378,468.36', '52,624,084.52'],
 ['金', None, None],
 [None, '', ''],
 ['支付的各项税费', '23,810,429.21', '45,883,304.77'],
 ['支付其他与经营活动有关的现金', '20,594,384.45', '30,234,338.65'],
 ['经营活动现金流出小计', '264,071,906.42', '248,975,632.82'],
 ['经营活动产生的现金流量净额', '-34,866,512.83', '27,518,354.57'],
 ['二、投资活动产生的现金流量：', '', ''],
 ['收回投资收到的现金', '183,000,000.00', '186,260,000.00'],
 ['取得投资收益收到的现金', '1,668,182.98', '2,763,508.99'],
 ['处置固定资产、无形资产和其他', '', ''],
 [None, '', '30,000.00'],
 ['长期资产收回的现金净额', None, None],
 [None, '',

In [34]:
p3c = pdf.pages[page_num3_c]
table3c = p3c.extract_tables() #多个表格
table3c[0]



[['三、筹资活动产生的现金流量：', '', ''],
 ['吸收投资收到的现金', '13,170,000.00', ''],
 ['其中：子公司吸收少数股东投资', '', ''],
 [None, '', ''],
 ['收到的现金', None, None],
 [None, '', ''],
 ['取得借款收到的现金', '3,000,000.00', '20,212,721.09'],
 ['收到其他与筹资活动有关的现金', '', ''],
 ['筹资活动现金流入小计', '16,170,000.00', '20,212,721.09'],
 ['偿还债务支付的现金', '20,212,721.09', '228,056,443.14'],
 ['分配股利、利润或偿付利息支付', '', ''],
 [None, '19,057,600.71', '3,743,099.97'],
 ['的现金', None, None],
 [None, '', ''],
 ['其中：子公司支付给少数股东的', '', ''],
 [None, '', ''],
 ['股利、利润', None, None],
 [None, '', ''],
 ['支付其他与筹资活动有关的现金', '', ''],
 ['筹资活动现金流出小计', '39,270,321.80', '231,799,543.11'],
 ['筹资活动产生的现金流量净额', '-23,100,321.80', '-211,586,822.02'],
 ['四、汇率变动对现金及现金等价物的', '', ''],
 [None, '', ''],
 ['影响', None, None],
 [None, '', ''],
 ['五、现金及现金等价物净增加额', '-52,989,301.89', '-196,777,103.39'],
 ['加：期初现金及现金等价物余额', '307,570,494.08', '467,183,100.89'],
 ['六、期末现金及现金等价物余额', '254,581,192.19', '270,405,997.50']]

In [35]:
# 把表格装进dataframe
df3a = pd.DataFrame(table3a, columns=table3a[0])
if np.size(table3c) < 3:
    df3c = pd.DataFrame(table3c[0], columns=table3c[0][0])
else: 
    df3c = pd.DataFrame(table3c, columns=table3c[0])


df3c

,三、筹资活动产生的现金流量：,,
0,三、筹资活动产生的现金流量：,,
1,吸收投资收到的现金,"13,170,000.00",
2,其中：子公司吸收少数股东投资,,
3,None,,
4,收到的现金,None,None
5,None,,
6,取得借款收到的现金,"3,000,000.00","20,212,721.09"
7,收到其他与筹资活动有关的现金,,
8,筹资活动现金流入小计,"16,170,000.00","20,212,721.09"
9,偿还债务支付的现金,"20,212,721.09","228,056,443.14"


In [64]:
# 还是洗数据

#查找数据各在第几行：

dfsize3a = df3a.shape[0]
dfsize3c = df3c.shape[0]

# 找经营活动产生的现金流量净额
for i in range(0,dfsize3a):
    if df3a.iloc[i][0] == "经营活动产生的现金流量净额":
        loc3a = i
        break
opcash_this = float(df3a.iloc[loc3a][1].replace(',',''))
opcash_last = float(df3a.iloc[loc3a][2].replace(',',''))

# 找筹资活动产生的现金流量净额
for i in range(0,dfsize3c):
    if df3c.iloc[i][0] == "筹资活动产生的现金流量净额":
        loc3c = i
        break
fincash_this = float(df3c.iloc[loc3c][1].replace(',',''))
fincash_last = float(df3c.iloc[loc3c][2].replace(',',''))

# 找投资活动产生的现金流量净额

if g == True: # 投资和经营在一张表上
    
    for i in range(0,dfsize3a):
        if df3a.iloc[i][0] == "投资活动产生的现金流量净额":
            loc3b = i
            break
 
    invcash_this = float(df3a.iloc[loc3b][1].replace(',',''))
    invcash_last = float(df3a.iloc[loc3b][2].replace(',',''))
            
elif g == False: # 投资和筹资在一张表上
    for i in range(0,dfsize3c):
        if df3c.iloc[i][0] == "投资活动产生的现金流量净额":
            loc3b = i
            break
    invcash_this = float(df3c.iloc[loc3b][1].replace(',',''))
    invcash_last = float(df3c.iloc[loc3b][2].replace(',',''))
    



In [67]:
loc3c

20

In [68]:

# 计算同比增减

opcash_var = round(100*(opcash_this - opcash_last)/opcash_last,2)
invcash_var = round(100*(invcash_this - invcash_last)/invcash_last,2)
fincash_var = round(100*(fincash_this - fincash_last)/fincash_last,2)



# 判断增减
updown_cash={}

if opcash_this < 0 and opcash_last < 0 and abs(opcash_this) < abs(opcash_last) :
    updown_cash[0] = '净流出同比减少'
elif opcash_this < 0 and opcash_last < 0 and abs(opcash_this) > abs(opcash_last):
    updown_cash[0] = '净流出同比增加'
elif opcash_this < 0 and opcash_last > 0:
    updown_cash[0] = '净流出同比增加'
elif opcash_this > 0 and opcash_last < 0:
    updown_cash[0] = '净流出同比减少'

    
if invcash_this < 0 and invcash_last < 0 and abs(invcash_this) < abs(invcash_last) :
    updown_cash[1] = '净流出同比减少'
elif invcash_this < 0 and invcash_last < 0 and abs(invcash_this) > abs(invcash_last):
    updown_cash[1] = '净流出同比增加'
elif invcash_this < 0 and invcash_last > 0:
    updown_cash[1] = '净流出同比增加'
elif invcash_this > 0 and invcash_last < 0:
    updown_cash[1] = '净流出同比减少'

    
if fincash_this < 0 and fincash_last < 0 and abs(fincash_this) < abs(fincash_last) :
    updown_cash[2] = '净流出同比减少'
elif fincash_this < 0 and fincash_last < 0 and abs(fincash_this) > abs(fincash_last):
    updown_cash[2] = '净流出同比增加'
elif fincash_this < 0 and fincash_last > 0:
    updown_cash[2] = '净流出同比增加'
elif fincash_this > 0 and fincash_last < 0:
    updown_cash[2] = '净流出同比减少'


liu_this = {}
liu_last = {}
n = 0
m = 0
for i in [opcash_this,invcash_this,fincash_this]:
    if i < 0:
        liu_this[n] = "流出"
    else:
        liu_this[n] = "流入"
    n = n + 1

for i in [opcash_last,invcash_last,fincash_last]:
    if i < 0:
        liu_last[m] = "流出"
    else:
        liu_last[m] = "流入"
    m = m + 1

In [69]:
cashline1 =  "经营活动现金净" +liu_this[0]+ mc(abs(opcash_this)) + ',' + updown_cash[0] + str(abs(opcash_var)) + '%' + "，去年同期净" + liu_last[0] + mc(abs(opcash_last)) + '；'
cashline2 =  "投资活动现金净" +liu_this[1]+ mc(abs(invcash_this)) + ',' + updown_cash[1] + str(abs(invcash_var)) + '%' + "，去年同期净" + liu_last[1] + mc(abs(invcash_last)) + '；'
cashline3 =  "筹资活动现金净" +liu_this[2]+ mc(abs(fincash_this)) + ',' + updown_cash[2] + str(abs(fincash_var)) + '%' + "，去年同期净" + liu_last[2] + mc(abs(fincash_last)) + '；'


In [70]:
# 财务摘要输出
print(line1_h1)
print(line2_h1)
print(line3_h1)
print(line4_h1)
print(line1_q2)
print(line2_q2)
print(line3_q2)
print(line4_q2)
print('\n')
#费用输出
print(rateline1)
print(rateline2)
print(rateline3)
print(rateline4)
print('\n')
#现金输出
print(cashline1)
print(cashline2)
print(cashline3)
print('\n')

20H1营业总收入1.96亿元,上年同期为2.04亿元,同比减少4.24%
20H1营业利润5153万元,上年同期为3976万元,同比增加29.6%
20H1归母净利润4546万元,上年同期为3518万元,同比增加29.23%
20H1扣非归母净利润4307万元,上年同期为2762万元,同比增加55.94%
20Q2营业总收入1.36亿元,上年同期为1.19亿元,同比增加13.96%
20Q2营业利润3762万元,上年同期为3308万元,同比增加13.71%
20Q2归母净利润3271万元,上年同期为2992万元,同比增加9.35%
20Q2扣非归母净利润3188万元,上年同期为2351万元,同比增加35.61%


销售费用1127万元，同比减少25.32%；销售费用率5.76%，较去年同期减少1.63pct；
管理费用1496万元，同比减少1.69%；管理费用率7.65%，较去年同期增加0.2pct；
研发费用2855万元，同比减少0.27%；研发费用率14.59%，较去年同期增加0.58pct；
财务费用-232万元，同比减少182.67%；财务费用率-1.19%，较去年同期减少2.57pct。


经营活动现金净流出3487万元,净流出同比增加226.7%，去年同期净流入2752万元；
投资活动现金净流入498万元,净流出同比减少139.17%，去年同期净流出1271万元；
筹资活动现金净流出2310万元,净流出同比减少89.08%，去年同期净流出2.12亿元；




SyntaxError: invalid character in identifier (<ipython-input-71-a1b1803b800f>, line 1)